__In this project I tried to explore a new approach to recommender systems which can utilize information in social networks, including user preference, item's general acceptance, and influence from social friends. I could achieve on par performance with standard CF based recommender systems. On the other hand, this system doesnt suffer from cold-start data sparsity problems common to CF based systems . This project uses real-world dataset from Yelp challenge which consists of  1637138 users and 108179 businesses.__




**1. Mounting  Yelp Data**

In [0]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


**2. Reading and storing json files**

In [0]:
with open('/gdrive/My Drive/Yelp_dataset/user.json') as f:
  lines_user = f.readlines() ;
with open('/gdrive/My Drive/Yelp_dataset/review.json') as f:
  lines_review = f.readlines() ;
with open('/gdrive/My Drive/Yelp_dataset/business.json') as f:
  lines_business = f.readlines() ; 
    
  
print(lines_user[0])
print(lines_review[0])
print(lines_business[9])



{"user_id":"l6BmjZMeQD3rDxWUbiAiow","name":"Rashmi","review_count":95,"yelping_since":"2013-10-08 23:11:33","useful":84,"funny":17,"cool":25,"elite":"2015,2016,2017","friends":"c78V-rj8NQcQjOI8KP3UEA, alRMgPcngYSCJ5naFRBz5g, ajcnq75Z5xxkvUSmmJ1bCg, BSMAmp2-wMzCkhTfq9ToNg, jka10dk9ygX76hJG0gfPZQ, dut0e4xvme7QSlesOycHQA, l4l5lBnK356zBua7B-UJ6Q, 0HicMOOs-M_gl2eO-zES4Q, _uI57wL2fLyftrcSFpfSGQ, T4_Qd0YWbC3co6WSMw4vxg, iBRoLWPtWmsI1kdbE9ORSA, xjrUcid6Ymq0DoTJELkYyw, GqadWVzJ6At-vgLzK_SKgA, DvB13VJBmSnbFXBVBsKmDA, vRP9nQkYTeNioDjtxZlVhg, gT0A1iN3eeQ8EMAjJhwQtw, 6yCWjFPtp_AD4x93WAwmnw, 1dKzpNnib-JlViKv8_Gt5g, 3Bv4_JxHXq-gVLOxYMQX0Q, ikQyfu1iViYh8T0us7wiFQ, f1GGltNaB7K5DR1jf3dOmg, tgeFUChlh7v8bZFVl2-hjQ, -9-9oyXlqsMG2he5xIWdLQ, Adj9fBPVJad8vSs-mIP7gw, Ce49RY8CKXVsTifxRYFTsw, M1_7TLi8CbdA89nFLlH4iw, wFsNv-hqbW_F5-IRqfBN6g, 0Q1L7zXHocaUZ2gsG2XJeg, cBFgmOCBdhYa0xoFEAzp_g, VrD_AgiFvzqtlR15vir3SQ, cpE-7HK514Sr5vpSen9CEQ, F1UYelhPFB-zIKlt0ygIZg, CQAL1hvsLMCzuJf9AglsXw, 1KnY1wr15WfEWIRLB9IS6g, QWFQ-kX

**Storing restaurant attributes like price range in dictionary **

In [0]:
import json

business = dict() ; 
for i in range (0, len(lines_business)):
  temp_line = json.loads(lines_business[i]);
  if ('attributes' in temp_line):
    if ((temp_line['attributes'] != None) and ('RestaurantsPriceRange2' in temp_line['attributes'])):
      business[temp_line['business_id']] = temp_line['attributes']['RestaurantsPriceRange2'];
#print(business);
print('business Map Done')
del lines_business ; 

business Map Done


Divide reviews into training and testing data and store them into dictionaries . 
- Note that dictionaries are used to avoid storing big matrices which will eat up the RAM . (Why ? )
- Consider this - There are 1637138 users and 108179 businesses , matrix size would blow up to 
 1637138*108179 and this would be silly because it would be an extremely sparse matrix. 




In [0]:
import random
from collections import defaultdict

reviews_train = defaultdict(dict)
reviews_b_train = defaultdict(dict)

reviews_test = defaultdict(dict)
reviews_b_test = defaultdict(dict)

for line in lines_review : 
    temp_line = json.loads(line);
    if (temp_line['business_id'] in business):
       if ([random.randint(0,1), random.randint(0,1)] == [1,1]):
        reviews_test[temp_line['user_id']][temp_line['business_id']] = temp_line['stars'];
        reviews_b_test[temp_line['business_id']][temp_line['user_id']] = temp_line['stars'];
       else:
        reviews_train[temp_line['user_id']][temp_line['business_id']] = temp_line['stars'];
        reviews_b_train[temp_line['business_id']][temp_line['user_id']] = temp_line['stars'];


print("No of training samples : ")
print(len(reviews_train))
print("No of testing samples : ")
print(len(reviews_test))

#print(len(reviews))
print('Review Map done')

del lines_review ; 

No of training samples : 
1193501
No of testing samples : 
611679
Review Map done


User Dictionary :

In [0]:
with open('/gdrive/My Drive/Yelp_dataset/user.json') as f:
  lines_user = f.readlines() ;
users = dict (); 
for line in lines_user:
  line = json.loads(line);
  users[line['user_id']] = line['friends']
print ('User Map done')
del lines_user ; 

User Map done


In [0]:
print("No of users   "+str(len(users)))
print("No of Reviews Train "+str(len(reviews_train)))
print("No of Reviews Test "+str(len(reviews_test)))
print("No of Business   "+str(len(business)))

No of users   1637138
No of Reviews Train 1193103
No of Reviews Test 611795
No of Business   108179


In [0]:
print('1st review : ' )
print(list(reviews_train.keys())[0], list(reviews_train[list(reviews_train.keys())[0]].keys())[0])

1st review : 
yXQM5uF2jS6es16SJzNHfg NZnhc2sEQy3RmzKTZnqtwQ


In [0]:
# Define function 
import numpy as np ; 
def get_user_rating_prob (UID):
  
    u_ratings = list(reviews_train[str(UID)].values())
   
    if (len (u_ratings) == 0 ):
      return [0.2]*5 ; 
    else:
      temp_hist = np.histogram(u_ratings, bins=[1, 2, 3, 4, 5, 6])[0]      
      return((temp_hist+1)/(temp_hist.sum()+5))

def get_business_rating_prob (BID):
  
    b_ratings = list(reviews_b_train[str(BID)].values())
   
    if (len (b_ratings) == 0 ):
      return np.array([0.2]*5) ; 
    else:
      temp_hist = np.histogram(b_ratings, bins=[1, 2, 3, 4, 5, 6])[0]      
      return((temp_hist+1)/(temp_hist.sum()+5))

# corrleation between two users - 1. Find businesses rated by both users  2. Find reatings correlation 
def get_user_similarity_hist (UID1, UID2):
   
   u1_bs = list(reviews_train[UID1].keys()) ; 
   u2_bs = list(reviews_train[UID2].keys()) ; 
   
   #print(u1_bs)
   #print(u2_bs)
    
   u1_rs =[]
   u2_rs =[]
    
   for item in u1_bs : 
      if item in u2_bs : 
        u1_rs.append(reviews_train[UID1][item]) ; 
        u2_rs.append(reviews_train[UID2][item]) ; 
   
   if (len(u1_rs) == 0):
      return np.array([])
   else:
      return(np.histogram( (np.array(u1_rs) - np.array(u2_rs)) , bins= [-4, -3, -2, -1, 0, 1, 2, 3, 4, 5])[0]) ; 

# Find friends of UID rating BID 
def find_friends_rating_same_restaurant ( UID , bid ): 
  
  friends = users[UID] ; 
  users_rating_bid = list(reviews_b_train[bid].keys())
  
  friends_of_UID_rating_bid = []
  
  for u in users_rating_bid : 
    if u in friends : 
       friends_of_UID_rating_bid.append(u)
  
  return(friends_of_UID_rating_bid)
  

def estimate_user_rating_based_on_item_attribute (UID, att):
  
  u_ratings = list(reviews_train[str(UID)].values())
  b_ids = list(reviews_train[str(UID)].keys())
  atts =[business[bid] for bid in b_ids];
  
  indices = [i for i, x in enumerate(atts) if x == att]
  
  #price range : 1 - 5 
  #print(atts)
  #print(att)
  #print(indices)
  
  if ( (len(u_ratings)== 0) or (len(atts) == 0)) :
    return np.array([0.2]*5) ; 
  else:
    temp_hist    = np.histogram(u_ratings, bins=[1, 2, 3, 4, 5, 6])[0]      
    att_ratings  = [u_ratings[i] for i in indices ]; 
    temp_att_hist= np.histogram(att_ratings, bins=[1, 2, 3, 4, 5, 6])[0]      
    second_term = np.divide ( (temp_att_hist + 1) , (temp_hist + 5));
    #print("second_term =")
    #print(second_term)
    return(second_term)

def normalize (array):
  array= array/ (array.sum()) ; 
  return(array);
  
#################################################################################
UID = list(reviews_test.keys())[0];
a = estimate_user_rating_based_on_item_attribute(UID, '2');
get_user_similarity_hist('yXQM5uF2jS6es16SJzNHfg', '6IAFtw20e4y99D2cA3jj9g')
# Test function 
a= list(reviews_test.keys())[1000]
print(a)
b = get_user_rating_prob(str(a))
print(b)
print("sum = "+str(sum(b)))

wCA6VAj6YlljD6OelSy-xQ
[0.2, 0.2, 0.2, 0.2, 0.2]
sum = 1.0


In [0]:
a=np.array([1,2,3,4])
b=np.array([6,8,10,4])
np.divide(a,b)

array([0.16666667, 0.25      , 0.3       , 1.        ])

In [0]:
import math
########## User Preference ########################
del P_mat
P_mat = dict();
i = 1; 
print("User preference : ")
for U_ratings in reviews_test.items() : 
    #print("i="+str(i))
    #if (i >3):
    #   break ; 
    i=i+1 ; 

    UID = U_ratings[0];
    user_rating_prob = get_user_rating_prob(UID); 
    for U_B_rating in U_ratings[1].items() : 
       att = business[U_B_rating[0]];
       #print("att = " +str(att))
       user_rating_att_prob = estimate_user_rating_based_on_item_attribute(UID, att); 
       user_rating_att_prob= normalize(user_rating_att_prob) ; 
       P_mat[UID, U_B_rating[0]] = normalize(np.multiply(user_rating_prob, user_rating_att_prob));
       #print("UID = " + str(UID));
       #print("BID = " + str(U_B_rating[0]));
       #print("user preference = " + str(P_mat[UID, U_B_rating[0]]));
print('------------------------  Complete -------------------------------------') 



User preference : 


In [0]:
########## Item Acceptance ########################  
print("Item Acceptance:  ")
Q_mat = dict() ; 
i= 1;
for B_ratings in reviews_b_test.items() : 
    if (i%10000 == 0):
      print(i)
    #if (i >3):
    #   break ; 
    i=i+1 ; 

    BID = B_ratings[0];
    business_rating_prob = get_business_rating_prob(BID); 
    #print(business_rating_prob)
    for B_U_rating in B_ratings[1].items() : 
       Q_mat[B_U_rating[0], BID] = normalize(business_rating_prob);
       #print("UID = " + str(B_U_rating[0]));
       #print("BID = " + str(BID));
       #print("Item Acceptance = " + str(Q_mat[B_U_rating[0], BID]));
print('------------------------  Complete -------------------------------------') 


Item Acceptance:  
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
------------------------  Complete -------------------------------------


In [0]:
########## Immediate Friend Influence ########################  

bias = 0.01 ; 

print("Friends Influence ")
R_mat = dict () ; 
i = 1; 
for U_ratings in reviews_test.items() : 
    #print("i= "+str(i))
    if (i % 10000 == 0):
      print(i)   
    i=i+1 ; 
    UID = U_ratings[0];
    
    for U_B_rating in U_ratings[1].items() : 
      bid = U_B_rating[0] ; 
      
      friends = find_friends_rating_same_restaurant(UID, bid) ; 
      #print("friends: ")
      #print(friends)
      pred = [] ; 
      
      for friend in friends : 
         if (friend in reviews_train):
            if (bid in reviews_train[friend]):
             
              
              friend_rating = reviews_train[friend][bid]; 
              hist_similarity = get_user_similarity_hist(UID, friend) ; 
              
              if (hist_similarity.size != 0 ):
                
                #friend_pred = friend_rating + np.multiply(np.array([-4, -3, -2, -1, 0 , 1, 2, 3, 4]) , (normalize(hist_similarity))).sum() ; 
                
                friends_prob = [0,0,0,0,0] ; 
                diff_array = [-4, -3, -2, -1, 0 , 1, 2, 3, 4];
                hist_similarity = normalize(hist_similarity)
                for el in diff_array : 
                  if ((el + friend_rating > 0) and (el + friend_rating < 6)):
                     friends_prob[el + round(friend_rating) -1 ] = hist_similarity[el+4]
                if (math.isnan(normalize(np.array(friends_prob))[0]) == False):
                  pred.append(normalize(np.array(friends_prob)))
                
      #print("predictions = ")
      #print(pred)
      
      if (len(pred) != 0 ):
        R_mat[UID, bid] = (np.array(pred).sum(0) / np.array(pred).shape[0]) ;
        R_mat[UID, bid] = (R_mat[UID, bid] + bias )/ np.sum(R_mat[UID, bid] + bias)
        
      else : 
        R_mat[UID, bid] = np.array([]) ;# Signature Number 
      
      #print("friends influence = ")
      #print(R_mat[UID, bid])
print('------------------------  Complete -------------------------------------') 
      

Friends Influence 


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:84: RuntimeWarning: invalid value encountered in true_divide


10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
------------------------  Complete -------------------------------------


In [0]:
import random

# check whether lenghts of all dictionaries are equal 
print('len of user preference dictionary : ' +str(len(P_mat.keys())))
print('len of Item acceptance dictionary : ' +str(len(Q_mat.keys())))
print('len of friends influence dictionary : ' +str(len(R_mat.keys())))


random_key = list(P_mat.keys())[random.randint(1, len(P_mat.keys()))]
print('\nRandom Key :')
print(random_key)
print('User preference : ')
print(P_mat[random_key])
print('Item Acceptance : ')
print(Q_mat[random_key])
print('Friends Influence : ')
print(R_mat[random_key])



len of user preference dictionary : 1362296
len of Item acceptance dictionary : 1362296
len of friends influence dictionary : 1362296

Random Key :
('jrqE4Jx-boDAxj4X7CLStQ', '0TBTV3q6QXCn9vNhy3Xi2w')
User preference : 
[0.00914606 0.08299205 0.14869727 0.58176911 0.1773955 ]
Item Acceptance : 
[0.02549575 0.03399433 0.09348442 0.40226629 0.44475921]
Friends Influence : 
[0.00952381 0.24761905 0.6047619  0.00952381 0.12857143]


In [0]:
import math
i=0
predicted_ratings = dict()
print('------------------------- Ratings prediction started --------------------------------')
for key in list(P_mat.keys()):
  i=i+1; 
  if(i % 25000 == 0):
    print(i)
  acc = np.array([0.0]*5) ; 
  if len(P_mat[key]) != 0 :
    #print(P_mat[key])
    acc += np.array(list(map(lambda x : math.log(x,10), P_mat[key]) ))
  if len(Q_mat[key]) != 0 :
    #print(Q_mat[key])
    acc += np.array(list(map(lambda x : math.log(x,10), Q_mat[key]) ))
  if len(R_mat[key]) != 0 :
    #print(R_mat[key])
    acc += np.array(list(map(lambda x : math.log(x,10), R_mat[key]) ))
  
  predicted_ratings[key] = np.argmax(acc)+1
  
print('-------------------- Prediction Complete --------------------------')
    

------------------------- Ratings prediction started --------------------------------
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
1010000
1020000
1030000
1040000
1050000
1060000
1070000
1080000
1090000
1100000
1110000
1120000
1130000
1140000
1150000
1160000
1170000
1180000
1190000
1200000
1210000
1220000
1230000
1240000
1250000
1260000
1270000
128000

In [0]:
RMSE = 0 
MAE = 0
for key1 in list(reviews_test.keys()):
  for key2 in list(reviews_test[key1].keys()):
    #print(key1)
    #print(key2)
    RMSE += (float(reviews_test[key1][key2])    -    float(predicted_ratings[key1, key2]))**2
    MAE  += (float(reviews_test[key1][key2])    -    float(predicted_ratings[key1, key2]-1))

RMSE = math.sqrt(RMSE / len(list(reviews_test.keys())))
MAE  = MAE / len(list(reviews_test.keys()))
print('------------------------------MAE = '+str(MAE)+'  ----------------------------------')

------------------------------MAE = 0.9847639759079551  ----------------------------------


**Following cells are just for my own debugging and experimentation , Please ignore them 





In [0]:
from collections import defaultdict
a = defaultdict(dict)
a[2][3]= 'N'
a[1][-4] = 'I'
a[-2][4]='K'
a[2][5]='H'

for item in a.items(): 
   print(item)

print(3 in a[2])

(2, {3: 'N', 5: 'H'})
(1, {-4: 'I'})
(-2, {4: 'K'})
True


In [0]:
for item in a.items(): 
  for item2 in item[1].items() :
     print(item2[1])

N
H
I
K


In [0]:
friends_prob = [0,0,0,0,0] ; 
friend_rating= 1; 
diff_array      = [-4,   -3,   -2,   -1,   0 ,     1,    2,   3,    4]; 
hist_similarity = [0.11, 0.21, 0.32, 0.43, 0.3 , 0.25, 0.12, 0.13, 0.14]; 
pred= []     
for el in diff_array : 
  if ((el + friend_rating > 0) and (el + friend_rating < 6)):
    print('el = '+str(el))
    
    friends_prob[el + friend_rating -1 ] = hist_similarity[el+4]

print(friends_prob)

pred.append(friends_prob)
pred.append(list(np.array(friends_prob)+0.1))

print(pred)

print(np.array(pred))
print(np.array(pred).sum(0) / np.array(pred).shape[0])



el = 0
el = 1
el = 2
el = 3
el = 4
[0.3, 0.25, 0.12, 0.13, 0.14]
[[0.3, 0.25, 0.12, 0.13, 0.14], [0.4, 0.35, 0.22, 0.23, 0.24000000000000002]]
[[0.3  0.25 0.12 0.13 0.14]
 [0.4  0.35 0.22 0.23 0.24]]
[0.35 0.3  0.17 0.18 0.19]
